In [1]:
import requests
import numpy as np
import pandas as pd
import time

In [25]:

ENGLISH_COUNTRY_CODE_LIST = ['US', 'CA', 'AU']
FRENCH_COUNTRY_CODE_LIST = ['FR']
TRENDING_TYPE_DICT = {
    'Now': None,
    'Music': '4gINGgt5dG1hX2NoYXJ0cw%3D%3D',
    'Gaming': '4gIcGhpnYW1pbmdfY29ycHVzX21vc3RfcG9wdWxhcg%3D%3D',
    'Movie': '4gIKGgh0cmFpbGVycw%3D%3D',
}

class YouTubeTrendingDataCollector:


    def __init__(self, country_code_list,trend_type_dict):
        self.country_code_list = country_code_list
        self.trend_type_dict = trend_type_dict
        self.data_dictionary = {
            "videoTitle": [],
            "videoId": [],
            "videoThumbnailUrl": [],
            "videoDescriptionSnippet": [],
            "videoRelativePublishedTimeText": [],
            "videoLength": [],
            "videoViewCountText": [],
            "videoCreatorName": [],
            "videoType": [],
            "trendingCountry": [],
            "exactViewNumber": [],
            "numberLikes": [],
            "videoDate": [],
            "creatorSubscriberNumber": [],
            "videoVerboseDescription": [],
            "numberOfComments": [],
            "isCreatorVerified": [],
            "videoKeywords": [],
            "videoLengthSeconds": [],
            "videoIsLiveContent": [],
            "videoCategory": [],
            "isFamilySafe": [],
            "videoExactPublishDate": [],
            "creatorUrl": []
        }

    def post_request_ytb(self):
        responses_dict = {}
        for country_code in self.country_code_list:
            responses_dict[country_code] = {}
            for trending_type, trending_type_url in self.trend_type_dict.items():
                json_data = self._create_json_data(country_code, trending_type_url)
                response = requests.post('https://www.youtube.com/youtubei/v1/browse', json=json_data)
                responses_dict[country_code][trending_type] = response.json()
                time.sleep(5)
        return responses_dict

    def _create_json_data(self, country_code, trending_type_url):
        return {
            'context': {
                'client': {
                    'gl': country_code,
                    'clientName': 'WEB',
                    'clientVersion': '2.20231115.01.01',
                    'originalUrl': 'https://www.youtube.com/feed/trending',
                },
            },
            'browseId': 'FEtrending',
            'params': trending_type_url,
        }

    def collect_data(self):
        response_dict = self.post_request_ytb()
        self._add_now_videos_shorts(response_dict)
        self._add_other_sections(response_dict)
        self.update_video_data()
        self.update_meta_data()
        return pd.DataFrame(self.data_dictionary)

    def _add_now_videos_shorts(self, response_dict):
        # Processing "Now", "Recently Trending" and "Shorts" videos
        for country_code in response_dict.keys():
            section_now = response_dict[country_code]['Now']["contents"]["twoColumnBrowseResultsRenderer"]["tabs"][0]["tabRenderer"]["content"]["sectionListRenderer"]["contents"]
            list_videos_before = section_now[0]["itemSectionRenderer"]["contents"][0]["shelfRenderer"]["content"]["expandedShelfContentsRenderer"]["items"]
            shorts_list = section_now[1]["itemSectionRenderer"]["contents"][0]["reelShelfRenderer"]["items"]
            list_videos_after = section_now[2]["itemSectionRenderer"]["contents"][0]["shelfRenderer"]["content"]["expandedShelfContentsRenderer"]["items"]
            list_videos_recently = section_now[3]["itemSectionRenderer"]["contents"][0]["shelfRenderer"]["content"]["expandedShelfContentsRenderer"]["items"]
            self._collect_video_data(list_videos_before, 'Now', country_code)
            self._collect_video_data(list_videos_after, 'Now', country_code)
            self._collect_video_data(list_videos_recently, 'Recently Trending', country_code)
            self._collect_short_data(shorts_list, country_code)

    def _add_other_sections(self, response_dict):
        # Processing "Music", "Gaming", "Movies" videos
        for country_code in response_dict.keys():
            for idx, key in enumerate(response_dict[country_code].keys()):
                if idx != 0:
                    sections = response_dict[country_code][key]["contents"]["twoColumnBrowseResultsRenderer"]["tabs"][idx]["tabRenderer"]["content"]["sectionListRenderer"]["contents"]
                    video_list = sections[0]["itemSectionRenderer"]["contents"][0]["shelfRenderer"]["content"]["expandedShelfContentsRenderer"]["items"]
                    self._collect_video_data(video_list, key, country_code)

    def _collect_video_data(self, video_list, trending_type, country_code):
        # Collecting video data
        nb_items = len(video_list)
        for k in range(nb_items):
            video_data = video_list[k]["videoRenderer"]
            self.data_dictionary["videoTitle"].append(video_data["title"]["runs"][0]["text"])
            self.data_dictionary["videoId"].append(video_data["videoId"])
            self.data_dictionary["videoThumbnailUrl"].append(video_data["thumbnail"]["thumbnails"][2]["url"])
            self.data_dictionary["videoDescriptionSnippet"].append(
                video_data.get("descriptionSnippet", {}).get("runs", [{}])[0].get('text', np.nan))
            self.data_dictionary["videoRelativePublishedTimeText"].append(
                video_data["publishedTimeText"]['simpleText'])
            self.data_dictionary["videoLength"].append(video_data["lengthText"]["simpleText"])
            self.data_dictionary["videoViewCountText"].append(video_data["viewCountText"]["simpleText"])
            self.data_dictionary["videoCreatorName"].append(video_data["ownerText"]["runs"][0]["text"])
            self.data_dictionary["videoType"].append(trending_type)
            self.data_dictionary["trendingCountry"].append(country_code)

    def _collect_short_data(self, shorts_list, country_code):
        # Collecting short video data
        for item in shorts_list:
            print(item)
            short_data = item["reelItemRenderer"]
            print(short_data.keys())
            self.data_dictionary["videoTitle"].append(short_data["headline"]["simpleText"])
            self.data_dictionary["videoId"].append(short_data["videoId"])
            self.data_dictionary["videoThumbnailUrl"].append(short_data["thumbnail"]["thumbnails"][0]["url"])
            self.data_dictionary["videoDescriptionSnippet"].append(np.nan)  # No description in shorts
            self.data_dictionary["videoRelativePublishedTimeText"].append(short_data["publishedTimeText"]["simpleText"])
            self.data_dictionary["videoLength"].append(np.nan)  # No length in shorts
            self.data_dictionary["videoViewCountText"].append(short_data["viewCountText"]["simpleText"])
            self.data_dictionary["videoCreatorName"].append(short_data["channelThumbnailSupportedRenderers"]["channelThumbnailWithLinkRenderer"]["channelName"]["simpleText"])
            self.data_dictionary["videoType"].append("Shorts")
            self.data_dictionary["trendingCountry"].append(country_code)

    def update_video_data(self):
        # Updating video data
        for video_id in self.data_dictionary["videoId"]:
            json_data = self._create_json_data_for_video(video_id)
            response = requests.post('https://www.youtube.com/youtubei/v1/next', json=json_data)
            video_response = response.json()
            self._process_individual_video_data(video_response, video_id)
            time.sleep(5)

    def _create_json_data_for_video(self, video_id):
        return {
            'context': {
                'client': {
                    'clientName': 'WEB',
                    'clientVersion': '2.20231208.01.00',
                },
            },
            'videoId': video_id,
        }

    def _process_individual_video_data(self, video_response, video_id):
        # Processing individual video data
        contents = video_response["contents"]["twoColumnWatchNextResults"]["results"]["results"]["contents"]
        primary_info = contents[0]["videoPrimaryInfoRenderer"]
        secondary_info = contents[1]["videoSecondaryInfoRenderer"]
        self.data_dictionary["exactViewNumber"].append(primary_info["viewCount"]["videoViewCountRenderer"]["viewCount"]["simpleText"])
        self.data_dictionary["numberLikes"].append(primary_info["videoActions"]["menuRenderer"]["topLevelButtons"][0]["segmentedLikeDislikeButtonViewModel"]["likeButtonViewModel"]["likeButtonViewModel"]["toggleButtonViewModel"]["toggleButtonViewModel"]["defaultButtonViewModel"]["buttonViewModel"]["title"])
        self.data_dictionary["videoDate"].append(primary_info["dateText"]["simpleText"])
        self.data_dictionary["creatorSubscriberNumber"].append(secondary_info["owner"]["videoOwnerRenderer"]["subscriberCountText"]["simpleText"])
        self.data_dictionary["videoVerboseDescription"].append(secondary_info.get("attributedDescription", {}).get("content", np.nan))
        self._process_comments_section(contents, video_id)
        self.data_dictionary["isCreatorVerified"].append(
            'badges' in secondary_info["owner"]["videoOwnerRenderer"] and 
            secondary_info["owner"]["videoOwnerRenderer"]["badges"][0]["metadataBadgeRenderer"]["tooltip"] == "Verified")


    def _process_comments_section(self, contents, video_id):
        # Processing comments section
        comments_section = next((item for item in contents if "commentsEntryPointHeaderRenderer" in item.get("itemSectionRenderer", {}).get("contents", [{}])[0]), None)
        if comments_section:
            self.data_dictionary["numberOfComments"].append(
                comments_section["itemSectionRenderer"]["contents"][0]["commentsEntryPointHeaderRenderer"]["commentCount"]["simpleText"]
            )
        else:
            self.data_dictionary["numberOfComments"].append(np.nan)

    def update_meta_data(self):
        # Updating metadata for individual videos
        for video_id in self.data_dictionary["videoId"]:
            json_data = self._create_json_data_for_metadata(video_id)
            response = requests.post('https://www.youtube.com/youtubei/v1/player', json=json_data)
            meta_response = response.json()
            self._process_metadata(meta_response)
            time.sleep(5)

    def _create_json_data_for_metadata(self, video_id):
        return {
            'context': {
                'client': {
                    'clientName': 'WEB',
                    'clientVersion': '2.20231208.01.00',
                },
            },
            'videoId': video_id,
        }

    def _process_metadata(self, meta_response):
        # Processing metadata
        video_details = meta_response["videoDetails"]
        player_microformat_renderer = meta_response["microformat"]["playerMicroformatRenderer"]
        self.data_dictionary["videoKeywords"].append(video_details.get("keywords", np.nan))
        self.data_dictionary["videoLengthSeconds"].append(video_details["lengthSeconds"])
        self.data_dictionary["videoIsLiveContent"].append(video_details["isLiveContent"])
        self.data_dictionary["videoCategory"].append(player_microformat_renderer["category"])
        self.data_dictionary["isFamilySafe"].append(player_microformat_renderer["isFamilySafe"])
        self.data_dictionary["creatorUrl"].append(player_microformat_renderer["ownerProfileUrl"])
        self.data_dictionary["videoExactPublishDate"].append(player_microformat_renderer["uploadDate"])




In [26]:
# Usage example
collector = YouTubeTrendingDataCollector(["FR"],TRENDING_TYPE_DICT)
df = collector.collect_data()
df.to_csv("./french_youtube_data.csv")

{'reelItemRenderer': {'videoId': 'yGk4Z-j1DMA', 'headline': {'simpleText': 'Que pense Squeezie de Cyprien ? @francetv'}, 'thumbnail': {'thumbnails': [{'url': 'https://i.ytimg.com/vi/yGk4Z-j1DMA/oar2.jpg?sqp=-oaymwEkCJUDENAFSFqQAgHyq4qpAxMIARUAAAAAJQAAyEI9AICiQ3gB&rs=AOn4CLBhkKBJbH7etobnvDcbyh7UAa9axg', 'width': 405, 'height': 720}], 'isOriginalAspectRatio': True}, 'viewCountText': {'accessibility': {'accessibilityData': {'label': '511K views'}}, 'simpleText': '511K views'}, 'navigationEndpoint': {'clickTrackingParams': 'CJcEEIf2BBgAIhMI9rS68_LEgwMVfcFJBx0LWwJS', 'commandMetadata': {'webCommandMetadata': {'url': '/shorts/yGk4Z-j1DMA', 'webPageType': 'WEB_PAGE_TYPE_SHORTS', 'rootVe': 37414}}, 'reelWatchEndpoint': {'videoId': 'yGk4Z-j1DMA', 'playerParams': '8AEByAO5AbgEOKIGFQHV2fo7BcIx1iIQlmCIkPmHvh1p0Q%3D%3D', 'thumbnail': {'thumbnails': [{'url': 'https://i.ytimg.com/vi/yGk4Z-j1DMA/frame0.jpg', 'width': 720, 'height': 1280}], 'isOriginalAspectRatio': True}, 'overlay': {'reelPlayerOverlay

KeyError: 'publishedTimeText'

In [2]:
TRENDING_TYPE_DICT = {
    'Now': None,
    'Music': '4gINGgt5dG1hX2NoYXJ0cw%3D%3D',
    'Gaming': '4gIcGhpnYW1pbmdfY29ycHVzX21vc3RfcG9wdWxhcg%3D%3D',
    'Movie': '4gIKGgh0cmFpbGVycw%3D%3D',
}

ENGLISH_COUNTRY_CODE_LIST = ['US', 'CA', 'AU']
FRENCH_COUNTRY_CODE_LIST = ['FR']

In [3]:
def post_request_ytb(country_code_list, trending_type_dict):
    responses_dict = {}
    for country_code in country_code_list:
        responses_dict[country_code] = {}
        for trending_type, trending_type_url in trending_type_dict.items():
            json_data = {
                'context': {
                    'client': {
                        'gl': country_code,
                        'clientName': 'WEB',
                        'clientVersion': '2.20231115.01.01',
                        'originalUrl': 'https://www.youtube.com/feed/trending',
                    },
                },
                'browseId': 'FEtrending',
                'params': trending_type_url,
            }

            response = requests.post(
                'https://www.youtube.com/youtubei/v1/browse',
                json=json_data,
            )
            responses_dict[country_code][trending_type] = response.json()
            time.sleep(5)
    return responses_dict        

In [4]:
def collect_video_data(video_list, data_dictionary, trending_type, country_code):
    # Collect data (generic videos) INPLACE
    nb_items = len(video_list)
    data_dictionary["videoTitle"] += [video_list[k]["videoRenderer"]["title"]["runs"][0]["text"] for k in range(nb_items)]
    data_dictionary["videoId"] += [video_list[k]["videoRenderer"]["videoId"] for k in range(nb_items)]
    data_dictionary["videoThumbnailUrl"] += [video_list[k]["videoRenderer"]["thumbnail"]["thumbnails"][2]["url"] for k in range(nb_items)]
    for k in range(nb_items):
        try:
            data_dictionary["videoDescriptionSnippet"].append(video_list[k]["videoRenderer"]["descriptionSnippet"]["runs"][0]['text'])
        except KeyError:
            data_dictionary["videoDescriptionSnippet"].append(np.nan)
    data_dictionary["videoRelativePublishedTimeText"] += [video_list[k]["videoRenderer"]["publishedTimeText"]['simpleText'] for k in range(nb_items)]
    data_dictionary["videoLength"] += [video_list[k]["videoRenderer"]["lengthText"]["simpleText"] for k in range(nb_items)]   
    data_dictionary["videoViewCountText"] += [video_list[k]["videoRenderer"]["viewCountText"]["simpleText"] for k in range(nb_items)]
    data_dictionary["videoCreatorName"] += [video_list[k]["videoRenderer"]["ownerText"]["runs"][0]["text"] for k in range(nb_items)]
    data_dictionary["videoType"] += [trending_type for k in range(nb_items)]
    data_dictionary["trendingCountry"] += [country_code for k in range(nb_items)]
    return None

In [5]:
def collect_short_data(shorts_list, data_dictionary, country_code):
    # Collect data (generic videos) INPLACE
    nb_items = len(shorts_list)
    data_dictionary["videoTitle"] += [shorts_list[k]["reelItemRenderer"]["headline"]["simpleText"] for k in range(nb_items)]
    data_dictionary["videoId"] += [shorts_list[k]["reelItemRenderer"]["videoId"] for k in range(nb_items)]
    data_dictionary["videoThumbnailUrl"] += [shorts_list[k]["reelItemRenderer"]["thumbnail"]["thumbnails"][0]["url"] for k in range(nb_items)]
    data_dictionary["videoDescriptionSnippet"] += [np.nan for k in range(nb_items)]
    data_dictionary["videoRelativePublishedTimeText"] += [np.nan for k in range(nb_items)]
    data_dictionary["videoLength"] += [np.nan for k in range(nb_items)]   
    data_dictionary["videoViewCountText"] += [shorts_list[k]["reelItemRenderer"]["viewCountText"]["simpleText"] for k in range(nb_items)]
    data_dictionary["videoCreatorName"] += [np.nan for k in range(nb_items)]
    data_dictionary["videoType"] += ['Short' for k in range(nb_items)]
    data_dictionary["trendingCountry"] += [country_code for k in range(nb_items)]
    return None

In [6]:
def add_now_videos_shorts(response_dict, data_dictionary):
    # Update data_dictionary with "Now", "Recently Trending" and "Shorts" videos data INPLACE
    for country_code in response_dict.keys():
        print("Current country : ", country_code)
        section_now = response_dict[country_code]['Now']["contents"]["twoColumnBrowseResultsRenderer"]["tabs"][0]["tabRenderer"]["content"]["sectionListRenderer"]["contents"]
        list_videos_before = section_now[0]["itemSectionRenderer"]["contents"][0]["shelfRenderer"]["content"]["expandedShelfContentsRenderer"]["items"]
        shorts_list = section_now[1]["itemSectionRenderer"]["contents"][0]["reelShelfRenderer"]["items"]
        list_videos_after = section_now[2]["itemSectionRenderer"]["contents"][0]["shelfRenderer"]["content"]["expandedShelfContentsRenderer"]["items"]
        list_videos_recently = section_now[3]["itemSectionRenderer"]["contents"][0]["shelfRenderer"]["content"]["expandedShelfContentsRenderer"]["items"]
        collect_video_data(list_videos_before, data_dictionary, 'Now', country_code)
        collect_video_data(list_videos_after, data_dictionary, 'Now', country_code)
        collect_video_data(list_videos_recently, data_dictionary, 'Recently Trending', country_code)
        collect_short_data(shorts_list, data_dictionary, country_code)
    return None

In [7]:
def add_other_sections(response_dict, data_dictionary):
    # Update data_dictionary with "Music", "Gaming", "Movies" videos data INPLACE
    for country_code in response_dict.keys():
        for idx, key in enumerate(response_dict[country_code].keys()): # in recent Python versions, dictionary are ordered
            if idx != 0:
                sections = response_dict[country_code][key]["contents"]["twoColumnBrowseResultsRenderer"]["tabs"][idx]["tabRenderer"]["content"]["sectionListRenderer"]["contents"]
                video_list = sections[0]["itemSectionRenderer"]["contents"][0]["shelfRenderer"]["content"]["expandedShelfContentsRenderer"]["items"]
                collect_video_data(video_list, data_dictionary, key, country_code)
    return None

In [8]:
def update_video_data(data_dictionary):
    # Update data for individual videos in data_dictionary INPLACE
    for video_id in data_dictionary["videoId"]:
        print("Currently processing video_id:", video_id)
        json_data = {
            'context': {
                'client': {
                    'clientName': 'WEB',
                    'clientVersion': '2.20231208.01.00',            
                    }
            },
            'videoId': video_id,
        }
        
        response = requests.post('https://www.youtube.com/youtubei/v1/next', json=json_data)
        video_response = response.json()
        individual_video_data = video_response["contents"]["twoColumnWatchNextResults"]["results"]["results"]["contents"]
        data_dictionary["exactViewNumber"].append(individual_video_data[0]["videoPrimaryInfoRenderer"]["viewCount"]["videoViewCountRenderer"]["viewCount"]["simpleText"])
        data_dictionary["numberLikes"].append(individual_video_data[0]["videoPrimaryInfoRenderer"]["videoActions"]["menuRenderer"]["topLevelButtons"][0]["segmentedLikeDislikeButtonViewModel"]["likeButtonViewModel"]["likeButtonViewModel"]["toggleButtonViewModel"]["toggleButtonViewModel"]["defaultButtonViewModel"]["buttonViewModel"]["title"])
        data_dictionary["videoDate"].append(individual_video_data[0]["videoPrimaryInfoRenderer"]["dateText"]["simpleText"])
        data_dictionary["creatorSubscriberNumber"].append(individual_video_data[1]["videoSecondaryInfoRenderer"]["owner"]["videoOwnerRenderer"]["subscriberCountText"]["simpleText"])
        try:
            data_dictionary["videoVerboseDescription"].append(individual_video_data[1]["videoSecondaryInfoRenderer"]["attributedDescription"]["content"])
        except KeyError: # no description available
            data_dictionary["videoVerboseDescription"].append(np.nan)
        try:    
            data_dictionary["numberOfComments"].append(individual_video_data[2]["itemSectionRenderer"]["contents"][0]["commentsEntryPointHeaderRenderer"]["commentCount"]["simpleText"])
        except KeyError: # correct for additional "merchandise" section
            try:
                data_dictionary["numberOfComments"].append(individual_video_data[3]["itemSectionRenderer"]["contents"][0]["commentsEntryPointHeaderRenderer"]["commentCount"]["simpleText"])
            except IndexError: # comments are turned off
                data_dictionary["numberOfComments"].append(np.nan)
        try:
            data_dictionary["isCreatorVerified"].append(individual_video_data[1]["videoSecondaryInfoRenderer"]["owner"]["videoOwnerRenderer"]["badges"][0]["metadataBadgeRenderer"]["tooltip"] == "Verified")
        except KeyError:
            data_dictionary["isCreatorVerified"].append(False)    
        time.sleep(5)
    return None

In [9]:
def update_meta_data(data_dictionary):
    # Update metadata for individual videos in data_dictionary INPLACE
    for video_id in data_dictionary["videoId"]:
        print("Currently processing video_id:", video_id)
        json_data = {
        'context': {
            'client': {
                'clientName': 'WEB',
                'clientVersion': '2.20231208.01.00',
            },
        },
        'videoId': video_id,
        }
        
        response = requests.post('https://www.youtube.com/youtubei/v1/player', json=json_data)
        
        meta_response = response.json()
        try:
            data_dictionary["videoKeywords"].append(meta_response["videoDetails"]["keywords"])
        except KeyError: # no keywords
            data_dictionary["videoKeywords"].append(np.nan)
        data_dictionary["videoLengthSeconds"].append(meta_response["videoDetails"]["lengthSeconds"])
        data_dictionary["videoIsLiveContent"].append(meta_response["videoDetails"]["isLiveContent"])
        data_dictionary["videoCategory"].append(meta_response["microformat"]["playerMicroformatRenderer"]["category"])
        data_dictionary["isFamilySafe"].append(meta_response["microformat"]["playerMicroformatRenderer"]["isFamilySafe"])
        data_dictionary["creatorUrl"].append(meta_response["microformat"]["playerMicroformatRenderer"]["ownerProfileUrl"])
        data_dictionary["videoExactPublishDate"].append(meta_response["microformat"]["playerMicroformatRenderer"]["uploadDate"])
        time.sleep(5)
    return None

#### Populate data_dictionary with trending videos

In [10]:
french_data_dictionary = {
"videoTitle": [],
"videoId": [],
"videoThumbnailUrl": [],
"videoDescriptionSnippet": [],
"videoRelativePublishedTimeText": [],
"videoLength": [],
"videoViewCountText": [],
"videoCreatorName": [],
"videoType": [],
"trendingCountry": [],
"exactViewNumber": [],
"numberLikes": [],
"videoDate": [],
"creatorSubscriberNumber": [],
"videoVerboseDescription": [],
"numberOfComments": [],
"isCreatorVerified": [],
"videoKeywords": [],
"videoLengthSeconds": [],
"videoIsLiveContent": [],
"videoCategory": [],
"isFamilySafe": [],
"videoExactPublishDate": [],
"creatorUrl": []
}

response_dict = post_request_ytb(FRENCH_COUNTRY_CODE_LIST, TRENDING_TYPE_DICT)
add_now_videos_shorts(response_dict, french_data_dictionary)
add_other_sections(response_dict, french_data_dictionary)

Current country :  FR


#### Update individual video data inside data_dictionary

In [11]:
update_video_data(french_data_dictionary)

Currently processing video_id: r7Bnj7--cUY
Currently processing video_id: cWWqr5ypRm4
Currently processing video_id: yWq4VArkd_8
Currently processing video_id: fTSuxn1jQkY
Currently processing video_id: HZAWQ97dTBE
Currently processing video_id: de6X9uwpKYM
Currently processing video_id: ebnDQr7JywI
Currently processing video_id: z4YkJFU1pls
Currently processing video_id: qLC58VZY5uQ
Currently processing video_id: D37b4PYN_EM
Currently processing video_id: f4R1CzyUBng
Currently processing video_id: i1OjDXZRTAE
Currently processing video_id: LPVaUKQQJsA
Currently processing video_id: I1AQLCuMIPk
Currently processing video_id: Mp8rBDsxj7Y
Currently processing video_id: mdIMXEZRdY8
Currently processing video_id: E45BpswQgKQ
Currently processing video_id: XZqXmswFXf8
Currently processing video_id: 91UCtGERwB4
Currently processing video_id: Is8MuKloD-c
Currently processing video_id: NzJGASY9FIo
Currently processing video_id: UvglTv8vxmc
Currently processing video_id: okC_EAqxKtM
Currently p

KeyError: 'videoPrimaryInfoRenderer'

#### Update individual video metadata inside data_dictionary

In [535]:
update_meta_data(french_data_dictionary)

Currently processing video_id: zePFAZMUnbU
Currently processing video_id: F8lPTlpThEo
Currently processing video_id: WB9wpwPVzyY
Currently processing video_id: P4hkzfYaz3Q
Currently processing video_id: 0_9I9Jn_YGE
Currently processing video_id: _Gv62FX9fFc
Currently processing video_id: dh8XxY717Gw
Currently processing video_id: s5GeFygLEig
Currently processing video_id: aVWrDes6A0Q
Currently processing video_id: tAcKfnf0zv4
Currently processing video_id: KlMrRUbmwPs
Currently processing video_id: XE_IK7ouDuc
Currently processing video_id: tVL9QrLQ0G0
Currently processing video_id: RwJ-OLVSNks
Currently processing video_id: OXBmgmCkIUc
Currently processing video_id: F_j29MEd2mY
Currently processing video_id: tmSm-tnifVk
Currently processing video_id: pBeB1s-58H8
Currently processing video_id: 22FwMc_V_Y4
Currently processing video_id: YfjpIw2pzbI
Currently processing video_id: GFIVMqa87QQ
Currently processing video_id: sEU5BTWnYk8
Currently processing video_id: FkY26NUZJyA
Currently p

In [537]:
df = pd.DataFrame(french_data_dictionary)

In [539]:
df.to_csv("./french_youtube_10_12_23.csv")